<a href="https://colab.research.google.com/github/Selma266/Malicious-URL-Detection/blob/main/Malicious_URL_Detection_2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Chargement Phishing URLs dataset + prétraitement

In [ ]:
import pandas as pd #la bibliothèque Pandas, utilisée pour la manipulation et l’analyse de données tabulaires.
df = pd.read_csv("/content/drive/MyDrive/Phishing URLs_cleaned.csv")  # read dataset url,label  #Chargement du dataset
df.head()

Suppression des lignes redondantes

In [ ]:
# BEFORE removing duplicates
print("Before removing duplicates:")
print("Shape:", df.shape)

# Step 2: Remove duplicate rows
df = df.drop_duplicates()   #Suppression des doublons

# AFTER removing duplicates
print("\nAfter removing duplicates:")
print("Shape:", df.shape)

Supprimer les NaN+ Conversion label en valeur Numeriques

In [ ]:
# Supprimer les lignes où url ou label est NaN
df = df.dropna(subset=["url", "label"]).reset_index(drop=True)  #reset_index(drop=True) Réinitialise l’index du DataFrame
# strip() supprime les espaces au début et à la fin URL
df["url"] = df["url"].str.strip()
# Conversion des labels texte en valeurs numériques
df["label"] = df["label"].map({"benign": 0, "malicious": 1}).astype(int) #Encodage des labels
print(df["label"].value_counts()) #Vérification de la distribution des classes Afichages

Creation des 3 distributions dataset1(50% 50%),dataset2(90% 10%)et dataset(10%-90%)

In [ ]:
#Génère les trois distributions expérimentales à partir du même dataset source.
import pandas as pd
# Sépare le dataset en deux sous-ensembles selon la classe (bénigne / malveillante).
benign = df[df['label'] == 0]
malicious = df[df['label'] == 1]
#Affiche le nombre d’échantillons par classe pour vérifier la disponibilité des données.
print(f"URLs bénignes disponibles: {len(benign)}")
print(f"URLs malveillantes disponibles: {len(malicious)}")
print(f"Total URLs disponibles: {len(df)}")

# Taille fixe pour tous les datasets\ to avoid 'Cannot take a larger sample than population error
TOTAL_SIZE = 176883

# ===== Dataset 1: Équilibré (50-50) =====
def create_balanced_dataset(benign, malicious, total_size=176883):
    """
    Crée un dataset équilibré avec 50% benign et 50% malicious
    """
    size_per_class = total_size // 2  #  176883 // 2 = 88441 pour chaque classe

    ben_sample = benign.sample(n=size_per_class, random_state=42)
    mal_sample = malicious.sample(n=size_per_class, random_state=42)

    dataset = pd.concat([ben_sample, mal_sample]) # Concatène  les deux DataFrames
    dataset = dataset.sample(frac=1, random_state=42).reset_index(drop=True)

    return dataset

# ===== Dataset 2: 90% bénignes, 10% malveillantes =====
def create_imbalanced_dataset_90_10(benign, malicious, total_size=176883):
    """
    Crée un dataset avec 90% benign et 10% malicious
    """
    ben_size = int(total_size * 0.9)
    mal_size = int(total_size * 0.1)

    ben_sample = benign.sample(n=ben_size, random_state=42)
    mal_sample = malicious.sample(n=mal_size, random_state=42)

    dataset = pd.concat([ben_sample, mal_sample])
    dataset = dataset.sample(frac=1, random_state=42).reset_index(drop=True)

    return dataset

# ===== Dataset 3: 10% bénignes, 90% malveillantes =====
def create_imbalanced_dataset_10_90(benign, malicious, total_size=176883):
    """
    Crée un dataset avec 10% benign et 90% malicious
    """
    ben_size = int(total_size * 0.1)
    mal_size = int(total_size * 0.9)

    ben_sample = benign.sample(n=ben_size, random_state=42)
    mal_sample = malicious.sample(n=mal_size, random_state=42)

    dataset = pd.concat([ben_sample, mal_sample])
    dataset = dataset.sample(frac=1, random_state=42).reset_index(drop=True)

    return dataset

# ===== Créer les 3 datasets =====
print("\n" + "="*60)
print("CRÉATION DES DATASETS")
print("="*60)

dataset_balanced = create_balanced_dataset(benign, malicious, TOTAL_SIZE)
dataset_imb_90_10 = create_imbalanced_dataset_90_10(benign, malicious, TOTAL_SIZE)
dataset_imb_10_90 = create_imbalanced_dataset_10_90(benign, malicious, TOTAL_SIZE)

# ===== Afficher les statistiques =====
print("\n📊 DATASET 1 - BALANCED (50-50):")
print(f"   Total samples: {len(dataset_balanced)}")
print(f"   Distribution:")
print(dataset_balanced['label'].value_counts().sort_index())
print(f"   Pourcentages:")
print(dataset_balanced['label'].value_counts(normalize=True).sort_index() * 100)

print("\n📊 DATASET 2 - IMBALANCED (90% benign - 10% malicious):")
print(f"   Total samples: {len(dataset_imb_90_10)}")
print(f"   Distribution:")
print(dataset_imb_90_10['label'].value_counts().sort_index())
print(f"   Pourcentages:")
print(dataset_imb_90_10['label'].value_counts(normalize=True).sort_index() * 100)

print("\n📊 DATASET 3 - IMBALANCED (10% benign - 90% malicious):")
print(f"   Total samples: {len(dataset_imb_10_90)}")
print(f"   Distribution:")
print(dataset_imb_10_90['label'].value_counts().sort_index())
print(f"   Pourcentages:")
print(dataset_imb_10_90['label'].value_counts(normalize=True).sort_index() * 100)


Charge le tokenizer RoBERTa pré-entraîné

In [ ]:
!pip install datasets

In [ ]:
import torch
from datasets import Dataset
from transformers import RobertaTokenizerFast

In [ ]:
#Charge le tokenizer RoBERTa pré-entraîné, qui convertit les URLs en tokens numériques exploitables par le modèle.
tokenizer = RobertaTokenizerFast.from_pretrained("roberta-base")  #Ce bloc charge le tokenizer RoBERTa, mais pas le modèle lui-même.
#Confirme le chargement correct du tokenizer et affiche la taille de son vocabulaire.
print("Tokenizer chargé avec succès!")
print(f"Taille du vocabulaire: {tokenizer.vocab_size}")

Définir la fonction de tokenisation

In [ ]:
# ÉTAPE 2 : Définir la fonction de tokenisation pour tokeniser un batch de données
def tokenize_batch(batch):
    return tokenizer(
        batch["url"],#le texte (URLs) à convertir en tokens
        truncation=True,  #Si le texte est trop long, on ne garde que les 128 premiers tokens
        padding="max_length",  #padding="max_length" Tous les textes doivent avoir la même longueur+remplit les textes courts jusqu’à max_length
        max_length=128 #limite les séquences à 128 tokens
    )

 convertir les 3 dataFrames pandas en HuggingFace Dataset, RoBERTa + Trainer ne travaillent pas directement avec pandas.

In [ ]:
#Conversion des datasets Pandas
hf_balanced = Dataset.from_pandas(dataset_balanced)
hf_90_10 = Dataset.from_pandas(dataset_imb_90_10)  #(90% benign - 10% malicious)
hf_10_90 = Dataset.from_pandas(dataset_imb_10_90)  #(10% benign - 90% malicious)

Appliquer le tokenizer RoBERTa aux 3 datasets, Obtenir des datasets prêts pour l’entraînement

In [ ]:
# Tokenisation du dataset équilibré (50–50)
hf_balanced = hf_balanced.map(
    tokenize_batch,#Applique la fonction tokenize_batch en batch sur le dataset Hugging Face.
    batched=True,#Traiter les données par lots (ex: 1000 lignes à la fois) au lieu d'une par une Taille du batch par défaut : 1000 lignes/ traitement vectorisé, plus rapide pour les grands datasets.
    remove_columns=["url"]# le modèle n’a besoin que des input_ids et attention_mask. on garde seulement les features nécessaires
)
#Taille du batch non précisée → Hugging Face utilise 1000 par défaut, mais peut être ajustée si GPU limité.

In [ ]:
#Tokenisation du dataset 10–90
hf_10_90 = hf_10_90.map(
    tokenize_batch,
    batched=True,
    remove_columns=["url"]
)

In [ ]:
#Tokenisation du dataset 90–10
hf_90_10 = hf_90_10.map(
    tokenize_batch,
    batched=True,
    remove_columns=["url"]
)

Définir le format PyTorch:Convertit les datasets HF en tenseurs PyTorch

In [ ]:
columns = ["input_ids", "attention_mask", "label"] #Spécifie les colonnes que le modèle RoBERTa utilisera pour l’entraînement
#Convertit les datasets HF en tenseurs PyTorch, prêts pour l’entraînement,  Ne garde que les colonnes spécifiées (input_ids, attention_mask, label) → optimisation mémoire.
hf_balanced.set_format(type="torch", columns=columns)
hf_90_10.set_format(type="torch", columns=columns)
hf_10_90.set_format(type="torch", columns=columns)

Affichage pour faire la vérification

In [ ]:
print(hf_balanced)
print(hf_90_10)
print(hf_10_90)

Séparation des données (TRAIN / TEST (80%-20%)) pour les 3 dataset

In [ ]:
#Sépare le dataset Hugging Face en train (80%) et test (20%) dataset 50%-50%
split_balanced = hf_balanced.train_test_split(test_size=0.2, seed=42)
train_balanced = split_balanced["train"]
test_balanced = split_balanced["test"]

# Vérification affiche le nombre d’échantillons pour train et test afin de vérifier le split.
print("Balanced Dataset")
print("Train size:", len(train_balanced))
print("Test size :", len(test_balanced))

In [ ]:
# dataset 90%-10%
split_90_10 = hf_90_10.train_test_split(test_size=0.2, seed=42)
train_90_10 = split_90_10["train"]
test_90_10 = split_90_10["test"]

# dataset 10%-90%
split_10_90 = hf_10_90.train_test_split(test_size=0.2, seed=42)
train_10_90 = split_10_90["train"]
test_10_90 = split_10_90["test"]
#Vérification similaire pour chacun (taille + distribution)

FINE-TUNING AVEC LoRA

Si tu entraînes le modèle sur balanced puis tu ré-entraîne sur 90/10 avec le même objet model, tu “contamines” l’expérience (carry-over).

➡️ Tu dois recréer un modèle neuf (mêmes poids init) pour chaque dataset.

In [ ]:
from transformers import RobertaForSequenceClassification
from peft import LoraConfig, get_peft_model, TaskType #outils pour appliquer LoRA
#Tokenizer:convertit le texte en tokens (input_ids, attention_mask) deja charger, On charge le modèle RoBERTa pour la classification binaire il reçoit ces tokens et produit des prédictions/ classification

def build_lora_roberta():
# Charger modèle pré-entraîné
    base = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=2) #Définit num_labels=2 pour la classification binaire (benign/malicious).
#la taille du modèle est grande (~125M paramètres) → GPU nécessaire pour fine-tuning complet, LoRA réduit ce coût.
#Paramètres standard pour fine-tuning efficace avec LoRA (Configuration LoRA )
    lora_config = LoraConfig(
        r=8, #rang de la décomposition low-rank → contrôle la capacité d’adaptation
        lora_alpha=16,
        lora_dropout=0.1,
        target_modules=["query", "value"],#appliqué uniquement aux matrices Q et V des transformers
        bias="none",
        task_type=TaskType.SEQ_CLS # Sequence Classification, tâche de classification de séquences
    )
    #  Appliquer LoRA au modèle,Transforme le modèle RoBERTa en modèle LoRA, où seules certaines couches sont entraînables.
    model = get_peft_model(base, lora_config)

    return model

Préparer l’entraînement

In [ ]:
from transformers import TrainingArguments
from transformers import Trainer

In [ ]:
# TrainingArguments : classe qui permet de définir tous les paramètres d’entraînement du modèle, Nombre d’epochs, taille des batches, learning rate, Stratégie d’évaluation et de sauvegarde GPU/FP16, logging, etc.
#Trainer : classe qui gère automatiquement l’entraînement, l’évaluation et la sauvegarde du modèle, automatisera tout l’entraînement : forward/backward pass, calcul de loss, optimisation, scheduler, checkpointing.
#Trainer appelle automatiquement  fonction compte metric à chaque évaluation (eval_dataset).
#output_dir doit être différent pour chaque expérience,Sinon tu écrases les résultats du balanced avec ceux du 90/10 etc.
training_args = TrainingArguments(
    output_dir="./results/balanced", # dossier pour sauvegarder les checkpoints
    num_train_epochs=1, #nombre de passages sur tout le dataset
    per_device_train_batch_size=16, # batch size pour l'entraînement
    per_device_eval_batch_size=32,# batch size pour évaluation
    learning_rate=5e-5,
    save_strategy="epoch", # sauvegarder modèle à chaque epoch
    eval_strategy="epoch", # Added evaluation strategy , évaluer à chaque fin d'epoch
    fp16=True  # activer float16 si GPU compatible
)

Fonction d’évaluation

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score
import numpy as np

In [ ]:
#calculer des métriques personnalisées (precision, recall, F1-score) pendant l’évaluation.
def compute_metrics(eval_pred):
    """
    Cette fonction est appelée par Trainer à chaque évaluation.
    eval_pred: tuple (logits, labels)
    Retourne: dictionnaire {"precision":..., "recall":..., "f1":...}
    """
    #Entrée : un tuple (logits, labels), Sortie : dictionnaire avec les métriques, compatible avec HF Trainer.
    logits, labels = eval_pred
    # Convertir logits en prédictions de classe
    preds = torch.argmax(torch.tensor(logits), axis=1).numpy()
    labels = labels.astype(int)

    # Calcul des métriques
    precision = precision_score(labels, preds)
    recall = recall_score(labels, preds)
    f1 = f1_score(labels, preds)

    return {
        "precision": precision,
        "recall": recall,
        "f1": f1
    }

In [ ]:
import torch
print(torch.cuda.is_available())  #entraînement sur GPU
#Cette fonction vérifie si CUDA est disponible sur ta machine. CUDA est la technologie de NVIDIA qui permet d’utiliser le GPU pour les calculs.


In [ ]:
if torch.cuda.is_available():
    print(torch.cuda.get_device_name(0))
else:
    print("Aucun GPU CUDA détecté")



#|# Matériel | Temps attendu (1 epoch) |
#| --------  | ----------------------- |
#| CPU      |  20–100 h                |
#| GPU T4   | ~30–60 min               |
#| GPU V100 | ~20–40 min               |
#| GPU A100 | ~5–10 min                |



Créer Trainer+ Lancer l'entraînement balanced dataset+calcul les metric

In [ ]:

# IMPORTANT: nouveau modèle pour chaque expérience
model_balanced = build_lora_roberta()
#Créer Trainer
trainer_balanced = Trainer(
   model= model_balanced,
    args=training_args,
    train_dataset=train_balanced, #  avec dataset équilibré
    eval_dataset=test_balanced,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)
# Lancer l'entraînement
trainer_balanced.train() # entraîne le modèle en appliquant LoRA sur les couches ciblées

In [ ]:
#évalue le modèle sur le dataset de test. Retourne un dictionnaire avec loss et métriques ( precision, recall, F1).
metrics = trainer_balanced.evaluate()  # évalue le modèle sur le dataset de test
#evaluate() est une méthode fournie par la classe Trainer de Hugging Face.
#Trainer fait automatiquement : Passe le modèle en mode évaluation (model.eval())

#Calcule :  Validation loss (automatique), Les métriques définies dans compute_metrics, Retourne un dictionnaire de résultats
print("Résultats finaux sur balanced dataset de test :")
print(metrics)

Lancer l'entraînement 90-10 dataset+ calcul les metric

In [ ]:
#change output_dir pour chaque experience +Toujours construire un modèle neuf
training_args = TrainingArguments(
    output_dir="./results/90-10-dataset", # dossier pour sauvegarder les checkpoints
    num_train_epochs=1, #nombre de passages sur tout le dataset
    per_device_train_batch_size=16, # batch size pour l'entraînement
    per_device_eval_batch_size=32,# batch size pour évaluation
    learning_rate=5e-5,
    save_strategy="epoch", # sauvegarder modèle à chaque epoch
    eval_strategy="epoch", # Added evaluation strategy , évaluer à chaque fin d'epoch
    fp16=True  # activer float16 si GPU compatible
)

In [ ]:
model_90_10=build_lora_roberta()
trainer_90_10 = Trainer(
   model= model_90_10,
    args=training_args,
    train_dataset=train_90_10,
    eval_dataset=test_90_10,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer_90_10.train()

In [ ]:
metrics = trainer_90_10.evaluate()
print("Résultats finaux sur 90-10 dataset de test :")
print(metrics)

Lancer l'entraînement 10-90 dataset+ calcul les metric

In [ ]:
training_args = TrainingArguments(
    output_dir="./results/10-90-dataset", # dossier pour sauvegarder les checkpoints
    num_train_epochs=1, #nombre de passages sur tout le dataset
    per_device_train_batch_size=16, # batch size pour l'entraînement
    per_device_eval_batch_size=32,# batch size pour évaluation
    learning_rate=5e-5,
    save_strategy="epoch", # sauvegarder modèle à chaque epoch
    eval_strategy="epoch", # Added evaluation strategy , évaluer à chaque fin d'epoch
    fp16=True  # activer float16 si GPU compatible
)

In [ ]:
model_10_90=build_lora_roberta()
trainer_10_90 = Trainer(
    model=model_10_90,
    args=training_args,
    train_dataset=train_10_90,
    eval_dataset=test_10_90,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer_10_90.train()

In [ ]:
metrics = trainer_10_90.evaluate()
print("Résultats finaux sur 10-90 dataset de test :")
print(metrics)

Application de smote sur embeddings+ RoBERTa LORA

In [ ]:
import torch
import numpy as np
from imblearn.over_sampling import SMOTE
from collections import Counter
from datasets import Dataset

# ========================================================================
# ÉTAPE 1 : EXTRAIRE LES EMBEDDINGS AVEC LE MODÈLE ROBERTA
# ========================================================================

def extract_embeddings(dataset, model_name="roberta-base", batch_size=32):
    """
    Extrait les embeddings RoBERTa pour tout le dataset

    Args:
        dataset: Dataset HuggingFace avec input_ids, attention_mask, label
        model_name: nom du modèle pour extraction d'embeddings
        batch_size: taille des batches pour l'extraction

    Returns:
        embeddings: np.array de shape (n_samples, embedding_dim)
        labels: np.array de shape (n_samples,)
    """
    from transformers import RobertaModel
    import torch
    from torch.utils.data import DataLoader

    print(f"\n{'='*60}")
    print("EXTRACTION DES EMBEDDINGS ROBERTA")
    print(f"{'='*60}")

    # Charger le modèle RoBERTa pour extraire les embeddings
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    roberta_model = RobertaModel.from_pretrained(model_name).to(device)
    roberta_model.eval()

    # Préparer le dataloader
    dataset_temp = dataset.remove_columns(['label'])
    dataloader = DataLoader(dataset_temp, batch_size=batch_size)

    embeddings_list = []

    print(f"📥 Extraction des embeddings en cours...")
    with torch.no_grad():
        for i, batch in enumerate(dataloader):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)

            # Obtenir les embeddings (on utilise le [CLS] token)
            outputs = roberta_model(
                input_ids=input_ids,
                attention_mask=attention_mask
            )

            # Prendre l'embedding du token [CLS] (première position)
            cls_embeddings = outputs.last_hidden_state[:, 0, :].cpu().numpy()
            embeddings_list.append(cls_embeddings)

            if (i + 1) % 100 == 0:
                print(f"   Traité: {(i + 1) * batch_size}/{len(dataset)} échantillons")

    # Concatener tous les embeddings
    embeddings = np.vstack(embeddings_list)
    labels = np.array(dataset['label'])

    print(f"✅ Extraction terminée!")
    print(f"   Shape embeddings: {embeddings.shape}")
    print(f"   Shape labels: {labels.shape}")

    # Libérer la mémoire GPU
    del roberta_model
    torch.cuda.empty_cache()

    return embeddings, labels

# ========================================================================
# ÉTAPE 2 : APPLIQUER SMOTE SUR LES EMBEDDINGS
# ========================================================================

def apply_smote_on_embeddings(embeddings, labels, random_state=42):
    """
    Applique SMOTE sur les embeddings RoBERTa

    Args:
        embeddings: np.array de shape (n_samples, embedding_dim)
        labels: np.array de shape (n_samples,)
        random_state: seed pour reproductibilité

    Returns:
        embeddings_resampled: embeddings après SMOTE
        labels_resampled: labels après SMOTE
    """
    print(f"\n{'='*60}")
    print("APPLICATION DE SMOTE SUR LES EMBEDDINGS")
    print(f"{'='*60}")

    # Distribution avant SMOTE
    print(f"\n📊 Distribution AVANT SMOTE:")
    counter_before = Counter(labels)
    print(f"   Classe 0 (benign): {counter_before[0]}")
    print(f"   Classe 1 (malicious): {counter_before[1]}")
    print(f"   Ratio: {counter_before[0]/counter_before[1]:.2f}:1")

    # Appliquer SMOTE
    print(f"\n🔄 Application de SMOTE sur les embeddings...")
    smote = SMOTE(random_state=random_state, k_neighbors=5)
    embeddings_resampled, labels_resampled = smote.fit_resample(embeddings, labels)

    # Distribution après SMOTE
    print(f"\n📊 Distribution APRÈS SMOTE:")
    counter_after = Counter(labels_resampled)
    print(f"   Classe 0 (benign): {counter_after[0]}")
    print(f"   Classe 1 (malicious): {counter_after[1]}")
    print(f"   Ratio: {counter_after[0]/counter_after[1]:.2f}:1")
    print(f"   Échantillons ajoutés: {len(labels_resampled) - len(labels)}")

    return embeddings_resampled, labels_resampled

# ========================================================================
# ÉTAPE 3 : CRÉER UN CUSTOM DATASET POUR ENTRAÎNEMENT
# ========================================================================

class EmbeddingDataset(torch.utils.data.Dataset):
    """
    Dataset personnalisé qui utilise directement les embeddings
    au lieu de passer par la tokenisation
    """
    def __init__(self, embeddings, labels):
        self.embeddings = torch.FloatTensor(embeddings)
        self.labels = torch.LongTensor(labels)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            'inputs_embeds': self.embeddings[idx],
            'labels': self.labels[idx]
        }

# ========================================================================
# ÉTAPE 4 : MODÈLE ROBERTA PERSONNALISÉ POUR EMBEDDINGS
# ========================================================================

from transformers import RobertaForSequenceClassification
from peft import LoraConfig, get_peft_model, TaskType

class RobertaForEmbeddingClassification(torch.nn.Module):
    """
    Wrapper pour RoBERTa qui accepte directement les embeddings
    """
    def __init__(self, model_name="roberta-base", num_labels=2):
        super().__init__()
        self.roberta = RobertaForSequenceClassification.from_pretrained(
            model_name,
            num_labels=num_labels
        )

    def forward(self, inputs_embeds, labels=None):
        # Ajouter une dimension pour la séquence (batch_size, 1, hidden_size)
        inputs_embeds = inputs_embeds.unsqueeze(1)

        outputs = self.roberta.roberta(inputs_embeds=inputs_embeds)
        sequence_output = outputs[0]

        # Prendre le premier token pour la classification
        logits = self.roberta.classifier(sequence_output[:, 0, :])

        loss = None
        if labels is not None:
            loss_fct = torch.nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, 2), labels.view(-1))

        return {'loss': loss, 'logits': logits} if loss is not None else {'logits': logits}

def build_lora_roberta_for_embeddings():
    """
    Crée un modèle RoBERTa avec LoRA adapté pour les embeddings
    """
    base = RobertaForEmbeddingClassification()

    lora_config = LoraConfig(
        r=8,
        lora_alpha=16,
        lora_dropout=0.1,
        target_modules=["query", "value"],
        bias="none",
        task_type=TaskType.SEQ_CLS
    )

    model = get_peft_model(base, lora_config)
    return model

# ========================================================================
# ÉTAPE 5 : CUSTOM TRAINER POUR EMBEDDINGS
# ========================================================================

from transformers import Trainer, TrainingArguments

class EmbeddingTrainer(Trainer):
    """
    Trainer personnalisé pour travailler avec des embeddings
    """
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.pop("labels")
        outputs = model(inputs_embeds=inputs['inputs_embeds'], labels=labels)
        loss = outputs['loss']
        return (loss, outputs) if return_outputs else loss

# ========================================================================
# ÉTAPE 6 : PIPELINE COMPLET AVEC SMOTE SUR EMBEDDINGS
# ========================================================================

print("\n" + "="*70)
print("🚀 PIPELINE COMPLET : SMOTE SUR EMBEDDINGS + ROBERTA LORA")
print("="*70)

# -------------------------------------------------------------------------
# 6.1 : BALANCED DATASET + SMOTE SUR EMBEDDINGS
# -------------------------------------------------------------------------

print("\n" + "="*70)
print("📊 Dataset 1/3: BALANCED (50-50) + SMOTE")
print("="*70)

# Extraire embeddings
embeddings_balanced, labels_balanced = extract_embeddings(train_balanced, batch_size=32)

# Appliquer SMOTE
embeddings_balanced_smote, labels_balanced_smote = apply_smote_on_embeddings(
    embeddings_balanced, labels_balanced, random_state=42
)

# Créer les datasets
train_emb_balanced = EmbeddingDataset(embeddings_balanced_smote, labels_balanced_smote)

# Extraire embeddings pour le test (sans SMOTE)
embeddings_test_balanced, labels_test_balanced = extract_embeddings(test_balanced, batch_size=32)
test_emb_balanced = EmbeddingDataset(embeddings_test_balanced, labels_test_balanced)

# Entraîner le modèle
print("\n🏋️ Entraînement du modèle...")

training_args = TrainingArguments(
    output_dir="./results/balanced_emb_smote",
    num_train_epochs=3,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=64,
    learning_rate=2e-5,
    save_strategy="epoch",
    eval_strategy="epoch",
    fp16=True,
    logging_steps=100,
    load_best_model_at_end=True,
    metric_for_best_model="f1"
)

model_balanced_smote = build_lora_roberta_for_embeddings()

trainer_balanced_smote = EmbeddingTrainer(
    model=model_balanced_smote,
    args=training_args,
    train_dataset=train_emb_balanced,
    eval_dataset=test_emb_balanced,
    compute_metrics=compute_metrics
)

trainer_balanced_smote.train()

# Évaluation
metrics_balanced_smote = trainer_balanced_smote.evaluate()
print("\n📊 Résultats BALANCED + SMOTE (embeddings):")
print(f"   Precision: {metrics_balanced_smote['eval_precision']:.4f}")
print(f"   Recall: {metrics_balanced_smote['eval_recall']:.4f}")
print(f"   F1-Score: {metrics_balanced_smote['eval_f1']:.4f}")

# -------------------------------------------------------------------------
# 6.2 : 90-10 DATASET + SMOTE SUR EMBEDDINGS
# -------------------------------------------------------------------------

print("\n" + "="*70)
print("📊 Dataset 2/3: 90-10 IMBALANCED + SMOTE")
print("="*70)

# Extraire embeddings
embeddings_90_10, labels_90_10 = extract_embeddings(train_90_10, batch_size=32)

# Appliquer SMOTE
embeddings_90_10_smote, labels_90_10_smote = apply_smote_on_embeddings(
    embeddings_90_10, labels_90_10, random_state=42
)

# Créer les datasets
train_emb_90_10 = EmbeddingDataset(embeddings_90_10_smote, labels_90_10_smote)

# Extraire embeddings pour le test
embeddings_test_90_10, labels_test_90_10 = extract_embeddings(test_90_10, batch_size=32)
test_emb_90_10 = EmbeddingDataset(embeddings_test_90_10, labels_test_90_10)

# Entraîner le modèle
print("\n🏋️ Entraînement du modèle...")

training_args = TrainingArguments(
    output_dir="./results/90_10_emb_smote",
    num_train_epochs=3,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=64,
    learning_rate=2e-5,
    save_strategy="epoch",
    eval_strategy="epoch",
    fp16=True,
    logging_steps=100,
    load_best_model_at_end=True,
    metric_for_best_model="f1"
)

model_90_10_smote = build_lora_roberta_for_embeddings()

trainer_90_10_smote = EmbeddingTrainer(
    model=model_90_10_smote,
    args=training_args,
    train_dataset=train_emb_90_10,
    eval_dataset=test_emb_90_10,
    compute_metrics=compute_metrics
)

trainer_90_10_smote.train()

# Évaluation
metrics_90_10_smote = trainer_90_10_smote.evaluate()
print("\n📊 Résultats 90-10 + SMOTE (embeddings):")
print(f"   Precision: {metrics_90_10_smote['eval_precision']:.4f}")
print(f"   Recall: {metrics_90_10_smote['eval_recall']:.4f}")
print(f"   F1-Score: {metrics_90_10_smote['eval_f1']:.4f}")

# -------------------------------------------------------------------------
# 6.3 : 10-90 DATASET + SMOTE SUR EMBEDDINGS
# -------------------------------------------------------------------------

print("\n" + "="*70)
print("📊 Dataset 3/3: 10-90 IMBALANCED + SMOTE")
print("="*70)

# Extraire embeddings
embeddings_10_90, labels_10_90 = extract_embeddings(train_10_90, batch_size=32)

# Appliquer SMOTE
embeddings_10_90_smote, labels_10_90_smote = apply_smote_on_embeddings(
    embeddings_10_90, labels_10_90, random_state=42
)

# Créer les datasets
train_emb_10_90 = EmbeddingDataset(embeddings_10_90_smote, labels_10_90_smote)

# Extraire embeddings pour le test
embeddings_test_10_90, labels_test_10_90 = extract_embeddings(test_10_90, batch_size=32)
test_emb_10_90 = EmbeddingDataset(embeddings_test_10_90, labels_test_10_90)

# Entraîner le modèle
print("\n🏋️ Entraînement du modèle...")

training_args = TrainingArguments(
    output_dir="./results/10_90_emb_smote",
    num_train_epochs=3,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=64,
    learning_rate=2e-5,
    save_strategy="epoch",
    eval_strategy="epoch",
    fp16=True,
    logging_steps=100,
    load_best_model_at_end=True,
    metric_for_best_model="f1"
)

model_10_90_smote = build_lora_roberta_for_embeddings()

trainer_10_90_smote = EmbeddingTrainer(
    model=model_10_90_smote,
    args=training_args,
    train_dataset=train_emb_10_90,
    eval_dataset=test_emb_10_90,
    compute_metrics=compute_metrics
)

trainer_10_90_smote.train()

# Évaluation
metrics_10_90_smote = trainer_10_90_smote.evaluate()
print("\n📊 Résultats 10-90 + SMOTE (embeddings):")
print(f"   Precision: {metrics_10_90_smote['eval_precision']:.4f}")
print(f"   Recall: {metrics_10_90_smote['eval_recall']:.4f}")
print(f"   F1-Score: {metrics_10_90_smote['eval_f1']:.4f}")

# ========================================================================
# ÉTAPE 7 : TABLEAU COMPARATIF FINAL
# ========================================================================

print("\n" + "="*70)
print("📊 TABLEAU COMPARATIF COMPLET")
print("="*70)

import pandas as pd

results_final = pd.DataFrame({
    'Dataset': [
        'Balanced (50-50) - Original',
        'Balanced (50-50) - SMOTE Embeddings',
        '90-10 - Original',
        '90-10 - SMOTE Embeddings',
        '10-90 - Original',
        '10-90 - SMOTE Embeddings'
    ],
    'Precision': [
        0.9990,
        metrics_balanced_smote['eval_precision'],
        0.9949,
        metrics_90_10_smote['eval_precision'],
        0.9994,
        metrics_10_90_smote['eval_precision']
    ],
    'Recall': [
        0.9864,
        metrics_balanced_smote['eval_recall'],
        0.9800,
        metrics_90_10_smote['eval_recall'],
        0.9916,
        metrics_10_90_smote['eval_recall']
    ],
    'F1-Score': [
        0.9927,
        metrics_balanced_smote['eval_f1'],
        0.9874,
        metrics_90_10_smote['eval_f1'],
        0.9955,
        metrics_10_90_smote['eval_f1']
    ]
})

print("\n")
print(results_final.to_string(index=False))

print("\n" + "="*70)
print("✅ PIPELINE TERMINÉ AVEC SUCCÈS")
print("="*70)

In [ ]:
# ========================================================================
# EXTENSION DU CODE : APPLICATION DE SMOTE-TOMEK SUR EMBEDDINGS + ROBERTA LORA
# ========================================================================

import torch
import numpy as np
from imblearn.combine import SMOTETomek
from imblearn.under_sampling import TomekLinks
from collections import Counter
from datasets import Dataset

# ========================================================================
# ÉTAPE 1 : EXTRAIRE LES EMBEDDINGS AVEC LE MODÈLE ROBERTA
# ========================================================================

def extract_embeddings(dataset, model_name="roberta-base", batch_size=32):
    """
    Extrait les embeddings RoBERTa pour tout le dataset

    Args:
        dataset: Dataset HuggingFace avec input_ids, attention_mask, label
        model_name: nom du modèle pour extraction d'embeddings
        batch_size: taille des batches pour l'extraction

    Returns:
        embeddings: np.array de shape (n_samples, embedding_dim)
        labels: np.array de shape (n_samples,)
    """
    from transformers import RobertaModel
    import torch
    from torch.utils.data import DataLoader

    print(f"\n{'='*60}")
    print("EXTRACTION DES EMBEDDINGS ROBERTA")
    print(f"{'='*60}")

    # Charger le modèle RoBERTa pour extraire les embeddings
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    roberta_model = RobertaModel.from_pretrained(model_name).to(device)
    roberta_model.eval()

    # Préparer le dataloader
    dataset_temp = dataset.remove_columns(['label'])
    dataloader = DataLoader(dataset_temp, batch_size=batch_size)

    embeddings_list = []

    print(f"📥 Extraction des embeddings en cours...")
    with torch.no_grad():
        for i, batch in enumerate(dataloader):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)

            # Obtenir les embeddings (on utilise le [CLS] token)
            outputs = roberta_model(
                input_ids=input_ids,
                attention_mask=attention_mask
            )

            # Prendre l'embedding du token [CLS] (première position)
            cls_embeddings = outputs.last_hidden_state[:, 0, :].cpu().numpy()
            embeddings_list.append(cls_embeddings)

            if (i + 1) % 100 == 0:
                print(f"   Traité: {(i + 1) * batch_size}/{len(dataset)} échantillons")

    # Concatener tous les embeddings
    embeddings = np.vstack(embeddings_list)
    labels = np.array(dataset['label'])

    print(f"✅ Extraction terminée!")
    print(f"   Shape embeddings: {embeddings.shape}")
    print(f"   Shape labels: {labels.shape}")

    # Libérer la mémoire GPU
    del roberta_model
    torch.cuda.empty_cache()

    return embeddings, labels

# ========================================================================
# ÉTAPE 2 : APPLIQUER SMOTE-TOMEK SUR LES EMBEDDINGS
# ========================================================================

def apply_smote_tomek_on_embeddings(embeddings, labels, random_state=42):
    """
    Applique SMOTE-Tomek sur les embeddings RoBERTa

    SMOTE-Tomek combine :
    - SMOTE : Over-sampling de la classe minoritaire
    - Tomek Links : Nettoyage des paires ambiguës à la frontière

    Args:
        embeddings: np.array de shape (n_samples, embedding_dim)
        labels: np.array de shape (n_samples,)
        random_state: seed pour reproductibilité

    Returns:
        embeddings_resampled: embeddings après SMOTE-Tomek
        labels_resampled: labels après SMOTE-Tomek
    """
    print(f"\n{'='*60}")
    print("APPLICATION DE SMOTE-TOMEK SUR LES EMBEDDINGS")
    print(f"{'='*60}")

    # Distribution avant SMOTE-Tomek
    print(f"\n📊 Distribution AVANT SMOTE-Tomek:")
    counter_before = Counter(labels)
    print(f"   Classe 0 (benign): {counter_before[0]}")
    print(f"   Classe 1 (malicious): {counter_before[1]}")
    print(f"   Ratio: {counter_before[0]/counter_before[1]:.2f}:1")
    print(f"   Total: {len(labels)} échantillons")

    # Appliquer SMOTE-Tomek
    print(f"\n🔄 Application de SMOTE-Tomek sur les embeddings...")
    print(f"   Étape 1: SMOTE génère des échantillons synthétiques...")
    print(f"   Étape 2: Tomek Links nettoie les paires ambiguës...")

    smote_tomek = SMOTETomek(
        random_state=random_state,
        smote=None,  # Utilise les paramètres par défaut de SMOTE
        tomek=TomekLinks(sampling_strategy='auto')
    )

    embeddings_resampled, labels_resampled = smote_tomek.fit_resample(embeddings, labels)

    # Distribution après SMOTE-Tomek
    print(f"\n📊 Distribution APRÈS SMOTE-Tomek:")
    counter_after = Counter(labels_resampled)
    print(f"   Classe 0 (benign): {counter_after[0]}")
    print(f"   Classe 1 (malicious): {counter_after[1]}")
    print(f"   Ratio: {counter_after[0]/counter_after[1]:.2f}:1")
    print(f"   Total: {len(labels_resampled)} échantillons")

    # Statistiques de changement
    samples_added = len(labels_resampled) - len(labels)
    print(f"\n📈 Statistiques:")
    print(f"   Échantillons SMOTE générés: {counter_after[min(counter_before, key=counter_before.get)] - counter_before[min(counter_before, key=counter_before.get)]}")
    print(f"   Paires Tomek supprimées: {len(labels) + (counter_after[min(counter_before, key=counter_before.get)] - counter_before[min(counter_before, key=counter_before.get)]) - len(labels_resampled)}")
    print(f"   Variation nette: {samples_added:+d} échantillons")

    return embeddings_resampled, labels_resampled

# ========================================================================
# ÉTAPE 3 : CRÉER UN CUSTOM DATASET POUR ENTRAÎNEMENT
# ========================================================================

class EmbeddingDataset(torch.utils.data.Dataset):
    """
    Dataset personnalisé qui utilise directement les embeddings
    au lieu de passer par la tokenisation
    """
    def __init__(self, embeddings, labels):
        self.embeddings = torch.FloatTensor(embeddings)
        self.labels = torch.LongTensor(labels)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            'inputs_embeds': self.embeddings[idx],
            'labels': self.labels[idx]
        }

# ========================================================================
# ÉTAPE 4 : MODÈLE ROBERTA PERSONNALISÉ POUR EMBEDDINGS
# ========================================================================

from transformers import RobertaForSequenceClassification
from peft import LoraConfig, get_peft_model, TaskType

class RobertaForEmbeddingClassification(torch.nn.Module):
    """
    Wrapper pour RoBERTa qui accepte directement les embeddings
    """
    def __init__(self, model_name="roberta-base", num_labels=2):
        super().__init__()
        self.roberta = RobertaForSequenceClassification.from_pretrained(
            model_name,
            num_labels=num_labels
        )

    def forward(self, inputs_embeds, labels=None):
        # Ajouter une dimension pour la séquence (batch_size, 1, hidden_size)
        inputs_embeds = inputs_embeds.unsqueeze(1)

        outputs = self.roberta.roberta(inputs_embeds=inputs_embeds)
        sequence_output = outputs[0]

        # Prendre le premier token pour la classification
        logits = self.roberta.classifier(sequence_output[:, 0, :])

        loss = None
        if labels is not None:
            loss_fct = torch.nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, 2), labels.view(-1))

        return {'loss': loss, 'logits': logits} if loss is not None else {'logits': logits}

def build_lora_roberta_for_embeddings():
    """
    Crée un modèle RoBERTa avec LoRA adapté pour les embeddings
    """
    base = RobertaForEmbeddingClassification()

    lora_config = LoraConfig(
        r=8,
        lora_alpha=16,
        lora_dropout=0.1,
        target_modules=["query", "value"],
        bias="none",
        task_type=TaskType.SEQ_CLS
    )

    model = get_peft_model(base, lora_config)
    return model

# ========================================================================
# ÉTAPE 5 : CUSTOM TRAINER POUR EMBEDDINGS
# ========================================================================

from transformers import Trainer, TrainingArguments

class EmbeddingTrainer(Trainer):
    """
    Trainer personnalisé pour travailler avec des embeddings
    """
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.pop("labels")
        outputs = model(inputs_embeds=inputs['inputs_embeds'], labels=labels)
        loss = outputs['loss']
        return (loss, outputs) if return_outputs else loss

# ========================================================================
# ÉTAPE 6 : PIPELINE COMPLET AVEC SMOTE-TOMEK SUR EMBEDDINGS
# ========================================================================

print("\n" + "="*70)
print("🚀 PIPELINE COMPLET : SMOTE-TOMEK SUR EMBEDDINGS + ROBERTA LORA")
print("="*70)

# -------------------------------------------------------------------------
# 6.1 : BALANCED DATASET + SMOTE-TOMEK SUR EMBEDDINGS
# -------------------------------------------------------------------------

print("\n" + "="*70)
print("📊 Dataset 1/3: BALANCED (50-50) + SMOTE-TOMEK")
print("="*70)

# Extraire embeddings
embeddings_balanced, labels_balanced = extract_embeddings(train_balanced, batch_size=32)

# Appliquer SMOTE-Tomek
embeddings_balanced_smote_tomek, labels_balanced_smote_tomek = apply_smote_tomek_on_embeddings(
    embeddings_balanced, labels_balanced, random_state=42
)

# Créer les datasets
train_emb_balanced = EmbeddingDataset(embeddings_balanced_smote_tomek, labels_balanced_smote_tomek)

# Extraire embeddings pour le test (sans SMOTE-Tomek)
embeddings_test_balanced, labels_test_balanced = extract_embeddings(test_balanced, batch_size=32)
test_emb_balanced = EmbeddingDataset(embeddings_test_balanced, labels_test_balanced)

# Entraîner le modèle
print("\n🏋️ Entraînement du modèle...")

training_args = TrainingArguments(
    output_dir="./results/balanced_emb_smote_tomek",
    num_train_epochs=3,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=64,
    learning_rate=2e-5,
    save_strategy="epoch",
    eval_strategy="epoch",
    fp16=True,
    logging_steps=100,
    load_best_model_at_end=True,
    metric_for_best_model="f1"
)

model_balanced_smote_tomek = build_lora_roberta_for_embeddings()

trainer_balanced_smote_tomek = EmbeddingTrainer(
    model=model_balanced_smote_tomek,
    args=training_args,
    train_dataset=train_emb_balanced,
    eval_dataset=test_emb_balanced,
    compute_metrics=compute_metrics
)

trainer_balanced_smote_tomek.train()

# Évaluation
metrics_balanced_smote_tomek = trainer_balanced_smote_tomek.evaluate()
print("\n📊 Résultats BALANCED + SMOTE-TOMEK (embeddings):")
print(f"   Precision: {metrics_balanced_smote_tomek['eval_precision']:.4f}")
print(f"   Recall: {metrics_balanced_smote_tomek['eval_recall']:.4f}")
print(f"   F1-Score: {metrics_balanced_smote_tomek['eval_f1']:.4f}")

# -------------------------------------------------------------------------
# 6.2 : 90-10 DATASET + SMOTE-TOMEK SUR EMBEDDINGS
# -------------------------------------------------------------------------

print("\n" + "="*70)
print("📊 Dataset 2/3: 90-10 IMBALANCED + SMOTE-TOMEK")
print("="*70)

# Extraire embeddings
embeddings_90_10, labels_90_10 = extract_embeddings(train_90_10, batch_size=32)

# Appliquer SMOTE-Tomek
embeddings_90_10_smote_tomek, labels_90_10_smote_tomek = apply_smote_tomek_on_embeddings(
    embeddings_90_10, labels_90_10, random_state=42
)

# Créer les datasets
train_emb_90_10 = EmbeddingDataset(embeddings_90_10_smote_tomek, labels_90_10_smote_tomek)

# Extraire embeddings pour le test
embeddings_test_90_10, labels_test_90_10 = extract_embeddings(test_90_10, batch_size=32)
test_emb_90_10 = EmbeddingDataset(embeddings_test_90_10, labels_test_90_10)

# Entraîner le modèle
print("\n🏋️ Entraînement du modèle...")

training_args = TrainingArguments(
    output_dir="./results/90_10_emb_smote_tomek",
    num_train_epochs=3,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=64,
    learning_rate=2e-5,
    save_strategy="epoch",
    eval_strategy="epoch",
    fp16=True,
    logging_steps=100,
    load_best_model_at_end=True,
    metric_for_best_model="f1"
)

model_90_10_smote_tomek = build_lora_roberta_for_embeddings()

trainer_90_10_smote_tomek = EmbeddingTrainer(
    model=model_90_10_smote_tomek,
    args=training_args,
    train_dataset=train_emb_90_10,
    eval_dataset=test_emb_90_10,
    compute_metrics=compute_metrics
)

trainer_90_10_smote_tomek.train()

# Évaluation
metrics_90_10_smote_tomek = trainer_90_10_smote_tomek.evaluate()
print("\n📊 Résultats 90-10 + SMOTE-TOMEK (embeddings):")
print(f"   Precision: {metrics_90_10_smote_tomek['eval_precision']:.4f}")
print(f"   Recall: {metrics_90_10_smote_tomek['eval_recall']:.4f}")
print(f"   F1-Score: {metrics_90_10_smote_tomek['eval_f1']:.4f}")

# -------------------------------------------------------------------------
# 6.3 : 10-90 DATASET + SMOTE-TOMEK SUR EMBEDDINGS
# -------------------------------------------------------------------------

print("\n" + "="*70)
print("📊 Dataset 3/3: 10-90 IMBALANCED + SMOTE-TOMEK")
print("="*70)

# Extraire embeddings
embeddings_10_90, labels_10_90 = extract_embeddings(train_10_90, batch_size=32)

# Appliquer SMOTE-Tomek
embeddings_10_90_smote_tomek, labels_10_90_smote_tomek = apply_smote_tomek_on_embeddings(
    embeddings_10_90, labels_10_90, random_state=42
)

# Créer les datasets
train_emb_10_90 = EmbeddingDataset(embeddings_10_90_smote_tomek, labels_10_90_smote_tomek)

# Extraire embeddings pour le test
embeddings_test_10_90, labels_test_10_90 = extract_embeddings(test_10_90, batch_size=32)
test_emb_10_90 = EmbeddingDataset(embeddings_test_10_90, labels_test_10_90)

# Entraîner le modèle
print("\n🏋️ Entraînement du modèle...")

training_args = TrainingArguments(
    output_dir="./results/10_90_emb_smote_tomek",
    num_train_epochs=3,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=64,
    learning_rate=2e-5,
    save_strategy="epoch",
    eval_strategy="epoch",
    fp16=True,
    logging_steps=100,
    load_best_model_at_end=True,
    metric_for_best_model="f1"
)

model_10_90_smote_tomek = build_lora_roberta_for_embeddings()

trainer_10_90_smote_tomek = EmbeddingTrainer(
    model=model_10_90_smote_tomek,
    args=training_args,
    train_dataset=train_emb_10_90,
    eval_dataset=test_emb_10_90,
    compute_metrics=compute_metrics
)

trainer_10_90_smote_tomek.train()

# Évaluation
metrics_10_90_smote_tomek = trainer_10_90_smote_tomek.evaluate()
print("\n📊 Résultats 10-90 + SMOTE-TOMEK (embeddings):")
print(f"   Precision: {metrics_10_90_smote_tomek['eval_precision']:.4f}")
print(f"   Recall: {metrics_10_90_smote_tomek['eval_recall']:.4f}")
print(f"   F1-Score: {metrics_10_90_smote_tomek['eval_f1']:.4f}")

# ========================================================================
# ÉTAPE 7 : TABLEAU COMPARATIF FINAL
# ========================================================================

print("\n" + "="*70)
print("📊 TABLEAU COMPARATIF COMPLET : ORIGINAL vs SMOTE-TOMEK")
print("="*70)

import pandas as pd

results_final = pd.DataFrame({
    'Dataset': [
        'Balanced (50-50) - Original',
        'Balanced (50-50) - SMOTE-TOMEK',
        '90-10 - Original',
        '90-10 - SMOTE-TOMEK',
        '10-90 - Original',
        '10-90 - SMOTE-TOMEK'
    ],
    'Precision': [
        0.9990,
        metrics_balanced_smote_tomek['eval_precision'],
        0.9949,
        metrics_90_10_smote_tomek['eval_precision'],
        0.9994,
        metrics_10_90_smote_tomek['eval_precision']
    ],
    'Recall': [
        0.9864,
        metrics_balanced_smote_tomek['eval_recall'],
        0.9800,
        metrics_90_10_smote_tomek['eval_recall'],
        0.9916,
        metrics_10_90_smote_tomek['eval_recall']
    ],
    'F1-Score': [
        0.9927,
        metrics_balanced_smote_tomek['eval_f1'],
        0.9874,
        metrics_90_10_smote_tomek['eval_f1'],
        0.9955,
        metrics_10_90_smote_tomek['eval_f1']
    ]
})

print("\n")
print(results_final.to_string(index=False))

# ========================================================================
# ÉTAPE 8 : ANALYSE DE L'IMPACT DE SMOTE-TOMEK
# ========================================================================

print("\n" + "="*70)
print("📈 ANALYSE DE L'IMPACT DE SMOTE-TOMEK")
print("="*70)

print("\n🔍 Impact sur Balanced (50-50):")
delta_prec = (metrics_balanced_smote_tomek['eval_precision'] - 0.9990) * 100
delta_rec = (metrics_balanced_smote_tomek['eval_recall'] - 0.9864) * 100
delta_f1 = (metrics_balanced_smote_tomek['eval_f1'] - 0.9927) * 100
print(f"   ΔPrecision: {delta_prec:+.2f}%")
print(f"   ΔRecall: {delta_rec:+.2f}%")
print(f"   ΔF1-Score: {delta_f1:+.2f}%")

print("\n🔍 Impact sur 90-10 Imbalanced:")
delta_prec = (metrics_90_10_smote_tomek['eval_precision'] - 0.9949) * 100
delta_rec = (metrics_90_10_smote_tomek['eval_recall'] - 0.9800) * 100
delta_f1 = (metrics_90_10_smote_tomek['eval_f1'] - 0.9874) * 100
print(f"   ΔPrecision: {delta_prec:+.2f}%")
print(f"   ΔRecall: {delta_rec:+.2f}%")
print(f"   ΔF1-Score: {delta_f1:+.2f}%")

print("\n🔍 Impact sur 10-90 Imbalanced:")
delta_prec = (metrics_10_90_smote_tomek['eval_precision'] - 0.9994) * 100
delta_rec = (metrics_10_90_smote_tomek['eval_recall'] - 0.9916) * 100
delta_f1 = (metrics_10_90_smote_tomek['eval_f1'] - 0.9955) * 100
print(f"   ΔPrecision: {delta_prec:+.2f}%")
print(f"   ΔRecall: {delta_rec:+.2f}%")
print(f"   ΔF1-Score: {delta_f1:+.2f}%")

print("\n" + "="*70)
print("💡 OBSERVATIONS CLÉS SUR SMOTE-TOMEK:")
print("="*70)
print("""
SMOTE-TOMEK combine deux techniques complémentaires:

1️⃣ SMOTE (Over-sampling):
   - Génère des échantillons synthétiques de la classe minoritaire
   - Interpole dans l'espace d'embeddings
   - Équilibre la distribution des classes

2️⃣ Tomek Links (Cleaning):
   - Identifie et supprime les paires d'échantillons ambiguës
   - Nettoie la frontière de décision
   - Améliore la séparabilité des classes

✅ Avantages sur SMOTE seul:
   - Meilleure qualité des frontières de décision
   - Réduction du bruit aux limites des classes
   - Amélioration potentielle de la précision
""")

print("\n" + "="*70)
print("✅ PIPELINE SMOTE-TOMEK TERMINÉ AVEC SUCCÈS")
print("="*70)


In [ ]:
Votre pipeline est correct pour répondre à :

“L’oversampling par SMOTE dans l’espace embedding améliore-t-il les performances en contexte déséquilibré ?”
Tester plusieurs ratios de classes (50/50, 90/10, 10/90)

✔️ C’est exactement ce qu’il faut faire pour étudier l’impact du déséquilibre.

Vous contrôlez :

la distribution des classes

le même type de données

la même tâche
→ Très bonne base expérimentale.
2) Ne jamais appliquer SMOTE sur le test

✔️ C’est une règle fondamentale respectée dans votre code.

3) Utiliser SMOTE sur des représentations continues

✔️ Très bon choix conceptuel.

SMOTE sur input_ids ❌

SMOTE sur embeddings ✔️
4) Comparer avec des métriques adaptées (Precision / Recall / F1)

✔️ Indispensable en contexte déséquilibré.


Votre code : transforme vos textes en embeddings RoBERTa, rééquilibre les classes avec SMOTE dans l’espace embedding, puis entraîne 3 modèles (un par distribution) et compare les performances.





In [ ]:
Voici, par points, ce que fait votre dernier code (SMOTE sur embeddings + RoBERTa LoRA), du début à la fin :

Importe les librairies

Charge PyTorch, NumPy, SMOTE (imblearn), Counter, etc.

Objectif : manipuler des tenseurs, extraire des embeddings, rééquilibrer des classes.

Définit une fonction pour extraire des embeddings RoBERTa (extract_embeddings)

Charge RobertaModel (l’encodeur RoBERTa).

Met le modèle sur GPU si disponible.

Parcourt le dataset par lots (DataLoader).

Pour chaque batch, calcule last_hidden_state.

Récupère l’embedding du premier token (token “CLS” / position 0) comme représentation du texte.

Concatène tous les embeddings en une matrice X de forme (N, hidden_size) et récupère y (labels).

Applique SMOTE sur les embeddings (apply_smote_on_embeddings)

Affiche la distribution des classes avant SMOTE (Counter).

Applique SMOTE dans l’espace continu des embeddings :

Génère des embeddings synthétiques pour la classe minoritaire.

Rééquilibre les classes (souvent vers 50/50).

Affiche la distribution après SMOTE et le nombre d’échantillons ajoutés.

Crée un Dataset PyTorch personnalisé (EmbeddingDataset)

Transforme les embeddings (float) et labels (int) en tenseurs PyTorch.

Permet au Trainer de lire les données sous la forme :

inputs_embeds = vecteur embedding

labels = classe

Définit un modèle de classification qui accepte directement des embeddings (RobertaForEmbeddingClassification)

Charge RobertaForSequenceClassification.

Dans forward() :

Ajoute une dimension “séquence” (transforme (batch, hidden) en (batch, 1, hidden)).

Fait passer ces embeddings via l’encodeur RoBERTa (self.roberta.roberta(inputs_embeds=...)).

Prend la sortie du “premier token” et la passe dans la couche classifier.

Calcule la loss CrossEntropy si labels est fourni.

Retourne logits (et loss pendant l’entraînement).

Ajoute LoRA au modèle (build_lora_roberta_for_embeddings)

Crée le modèle précédent.

Configure LoRA (r, alpha, dropout, modules ciblés query et value).

“Wrap” le modèle avec PEFT/LoRA pour fine-tuning léger (moins de paramètres entraînés).

Crée un Trainer personnalisé (EmbeddingTrainer)

Redéfinit compute_loss pour appeler correctement le modèle avec :

inputs_embeds au lieu de input_ids

Récupère la loss retournée par le modèle, ce qui permet d’utiliser HuggingFace Trainer.

Lance le pipeline complet sur 3 jeux d’entraînement

Pour chaque dataset (Balanced, 90/10, 10/90), le code :

Extrait les embeddings du train

Applique SMOTE sur embeddings du train

Crée EmbeddingDataset pour train

Extrait les embeddings du test (sans SMOTE)

Crée EmbeddingDataset pour test

Définit TrainingArguments

Construit le modèle LoRA

Entraîne avec EmbeddingTrainer

Évalue et affiche Precision / Recall / F1

Génère un tableau comparatif final

Construit un DataFrame avec :

résultats “Original” (valeurs fixes)

résultats “SMOTE Embeddings” (mesurés)

Affiche le tableau pour comparer avant/après.

Termine le script

Affiche “pipeline terminé”.